In [66]:
import pandas as pd
import implicit
import os
import numpy as np
from scipy import sparse
import scipy
import lightfm
from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from tqdm import tqdm

Решается задача товарных рекомендаций.

- `purchases_train.csv` - история покупок в розничном магазине (с 21 октября 2003 года по 12 марта 2004 года)

- `purchases_test.csv` - покупки за следующую неделю (с 13 по 19 марта 2004 года). В этой выборке для каждого пользователя исключены товары, которые он уже покупал за период обучающей выборки

- `customers.csv` - пол клиентов

В решении ниже
- обучается модель матричного разложения AlternateLeastSquares
- сравнивается с тестовыми данными
- измеряется ее качество по метрике map@10
- данные о поле клиентов не используются

Улучшите решение с помощью информацию о поле клиентов

In [67]:
purchases_train = pd.read_csv('purchases_train.csv')
purchases_train.head()

,customer_id,product_id,datetime
0,8698595,12530,2004-03-10 22:18:43.497459200
1,13271885,7541,2004-03-06 02:24:43.209763200
2,16852746,13134,2004-03-10 01:03:09.598614400
3,16852746,6572,2004-03-04 16:45:16.522566400
4,14619070,4659,2004-03-12 13:29:35.011481600


Обучаем модель AlternateLeastSquares

In [68]:
user_items = sparse.coo_matrix(
    (
        np.ones(purchases_train.customer_id.size, dtype=np.float32),
        (
            purchases_train.customer_id,
            purchases_train.product_id
        )
    )
).tocsr()

In [69]:
item_users = user_items.T.tocsr()

In [70]:
model = implicit.als.AlternatingLeastSquares(factors=15, iterations=1)

In [71]:
np.random.seed(42)
model.fit(item_users=item_users)

purchases_test.csv содержит данные о покупках с 13 марта 2004 по 20 марта 2004 - то есть неделя следующая за обучающей выборкой

для каждого пользователя исключены те товары, которые он покупал в обучающей выборке

In [72]:
purchases_test = pd.read_csv('purchases_test.csv')
display(
    purchases_test.head(),
)

,customer_id,product_id,datetime
0,1021292,6197,2004-03-18 13:35:19.145152000
1,11379978,4659,2004-03-19 18:51:31.887936000
2,13271885,5659,2004-03-14 05:47:21.544166400
3,13271885,1015,2004-03-15 14:41:19.702089601
4,12315337,12072,2004-03-19 10:39:17.148105600


Измеряем качество рекомендаций с помощью метрики map@10

In [73]:
relevant = purchases_test.groupby('customer_id')['product_id'].apply(lambda s: s.values).reset_index()
relevant.rename(columns={'product_id': 'product_ids'}, inplace=True)
relevant.head()

,customer_id,product_ids
0,107,[5868]
1,453,[11854]
2,1011,"[10609, 7110]"
3,1135,[8994]
4,2947,[5868]


In [74]:
recommendations = []
for user_id in relevant['customer_id']:
    recommendations.append([x[0] for x in model.recommend(userid=user_id, user_items=user_items, N=10)])

In [75]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if len(actual) == 0:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [76]:
mapk(relevant['product_ids'], recommendations, k=10)

0.11926911588325763

Задание: используйте пол клиентов для улучшения модели

In [77]:
customers = pd.read_csv('customers.csv')
customers.head()

,customer_id,sex
0,14386819,Female
1,1481405,NaN
2,16745074,NaN
3,10325906,NaN
4,11167384,NaN


In [78]:
# -------------------------------------------------------------------------------------------------
# Для начала посчитаем среднее значение mapk за N обученных моделей и среднее время 1 обучения модели

In [79]:
import time
best_model = model
best_mapk = 0
model_test = pd.DataFrame(columns = ['N iter', 'mapk value'])
n = 10
mapk_mean = 0
mapk_sum = 0
startTime = time.time()
for iteration in tqdm(range(n)):
    model = implicit.als.AlternatingLeastSquares(factors=15, iterations=1, calculate_training_loss = True, regularization = 0.01)
    np.random.seed(42)
    model.fit(item_users=item_users)
    qur_model = model
    recommendations = []
    for user_id in relevant['customer_id']:
        recommendations.append([x[0] for x in qur_model.recommend(userid=user_id, user_items=user_items, N=10)])
    qur_mapk = mapk(relevant['product_ids'], recommendations, k=10)
    print(qur_mapk)
    mapk_sum+= qur_mapk
    if qur_mapk > best_mapk:
        best_model = qur_model
        best_mapk = qur_mapk
    model_test.loc[iteration] = [iteration, qur_mapk]
finishTime = time.time()
mapk_mean = mapk_sum / n
meanTime = (finishTime - startTime) / n
print('среднее время обучения:{} секунд'.format(round(meanTime,2)))
print('лучший показатель mapk:',best_mapk)
print('средний показатель mapk:',mapk_mean)

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:15<02:21, 15.77s/it]

0.06569403300863647


 20%|██        | 2/10 [00:31<02:06, 15.75s/it]

0.05830536676993863


...

0.10778429359087882


 90%|█████████ | 9/10 [02:28<00:17, 17.06s/it]

0.1015088071925121


100%|██████████| 10/10 [02:46<00:00, 17.31s/it]

0.11052181707616124
среднее время обучения:16.66 секунд
лучший показатель mapk: 0.11052181707616124
средний показатель mapk: 0.09187014991055702


In [15]:
#---------------------------Адаптируем изначальную модель-------------------------------
# Суть в том, что метод формирования разреженной матрицы sparse.coo_matrix рассчитывает размер матрицы 
# по максимальным значениям параметров, в данном случае индексов customer_id и product_id. Соответсвенно матрица,
# подаваемая на обучение алгоритму ALS, имеет размер (13'831, 21'696'338), среднее время обучения 
# с заданными параметрами (factors=15, iterations=1) составляет около 16 секунд(на моем ноутбуке), максимальное значение mapk
# получается не более 0.11, а среднее за 10 обучений не более 0.092.
# Что делаю: пронумировываю значения параметров customer_id и product_id от 0 и до последнего значения, т.е. 
# каждому реальному значению даю новое, состовляя сразу же таблицу соответсвий искомых значений новым.
# Размер матрицы на вход в модель падает до (1792, 107491), что равно количеству уникальных значений параметров.
# Далее, при обучении модели количество итераций ставлю равным 1, т.к. если в параметры модели добавить
# calculate_training_loss = True и понаблюдать за значениями на каждой итерации, то видно, что среднеквадратичные 
# потери модели очень быстро уходят в заоблачные значения, а модель, соответсвенно, переобучается, что видно на 
# значении mapk на тестовой выборке - оно приобретает парядок 10^(-5) степени. Ставя же значение в 1 итерацию на 
# обучение, получаем хорошие результаты - значение mapk увеличивается на 3-4 порядка.


In [16]:
# создаем матрицу оптимального размера

In [80]:
purchases_train = pd.read_csv('purchases_train.csv')
purchases_train = purchases_train.drop('datetime', axis = 1)
customer_ids = list(purchases_train.customer_id.unique())
product_ids = list(purchases_train.product_id.unique())
rows = purchases_train.customer_id.astype('category', categories=customer_ids).cat.codes
cols = purchases_train.product_id.astype('category', categories=product_ids).cat.codes
ones = np.ones(purchases_train.customer_id.size, dtype=np.float32)
purchases_train_csr = scipy.sparse.csr_matrix((ones, (rows, cols)), shape=(len(customer_ids), len(product_ids))).astype('float')

In [81]:
user_items = purchases_train_csr
item_users = user_items.T.tocsr()

In [82]:
# смотрим коэффициент разреженности

In [83]:
def calculate_sparsity(M):
    """
    Computes sparsity of matrix
    
    Parameters:
        M: matrix to be computed
    """
    matrix_size = float(M.shape[0]*M.shape[1]) # Number of possible interactions in the matrix
    num_plays = len(M.nonzero()[0]) # Number of items interacted with
    sparsity = 100*(1 - float(num_plays/matrix_size))
    return sparsity
calculate_sparsity(item_users)

99.82308942476247

In [84]:
# подготавливаем таблицу соответствий исходных и новых значений индексов параметров
# и заменяем в тестовой выборке индексы на новые 

In [85]:
customers_products_catcodes = pd.DataFrame(purchases_train)
customers_products_catcodes['customer_id_cats'] = rows
customers_products_catcodes['product_id_cats'] = cols
customers_products_catcodes.drop_duplicates(subset = ['customer_id'], inplace = True)

def customer_to_catcode(customer_id):
    out = customer_id
    if customer_id in customers_products_catcodes.customer_id.values:
        out = customers_products_catcodes[customers_products_catcodes.customer_id == customer_id].customer_id_cats.values[0]
    return out
def product_to_catcode(product_id):
    out = product_id
    if product_id in customers_products_catcodes.product_id.values:
        out = customers_products_catcodes[customers_products_catcodes.product_id == product_id].product_id_cats.values[0]
    return out

purchases_test = pd.read_csv('purchases_test.csv')
relevant = purchases_test.drop('datetime', axis = 1)
relevant.customer_id = relevant.customer_id.apply(customer_to_catcode)
relevant.product_id = relevant.product_id.apply(product_to_catcode)
relevant = relevant.groupby('customer_id')['product_id'].apply(lambda s: s.values).reset_index()
relevant.rename(columns={'product_id': 'product_id_cats'}, inplace=True)
relevant.rename(columns={'customer_id': 'customer_id_cats'}, inplace=True)
relevant.head()

,customer_id_cats,product_id_cats
0,0,[5]
1,1,"[191, 669, 282, 1190, 200, 856, 173, 1091, 10,..."
2,2,"[65, 505, 492, 808]"
3,3,"[245, 333, 147, 92, 404, 336]"
4,6,"[282, 10, 331, 791]"


In [86]:
customers_products_catcodes.head()

,customer_id,product_id,customer_id_cats,product_id_cats
0,8698595,12530,0,0
1,13271885,7541,1,1
2,16852746,13134,2,2
4,14619070,4659,3,4
5,12324374,12072,4,5


In [87]:
# создаем и обучаем модель

In [88]:
model = implicit.als.AlternatingLeastSquares(factors=15, iterations=1, calculate_training_loss = True, regularization = 0.01)

In [89]:
np.random.seed(42)
model.fit(item_users=item_users)

In [90]:
recommendations = []
for user_id in relevant['customer_id_cats']:
    recommendations.append([x[0] for x in model.recommend(userid=user_id, user_items=user_items, N=10)])

In [91]:
mapk(relevant['product_id_cats'], recommendations, k=10)

0.08362396657502297

In [92]:
# измеряем что получилось

In [93]:
best_model = model
best_mapk = 0
model_test = pd.DataFrame(columns = ['N iter', 'mapk value'])
n = 10
mapk_mean = 0
mapk_sum = 0
startTime = time.time()
for iteration in tqdm(range(n)):
    model = implicit.als.AlternatingLeastSquares(factors=15, iterations=1, calculate_training_loss = True, regularization = 0.01)
    np.random.seed(42)
    model.fit(item_users=item_users)
    qur_model = model
    recommendations = []
    for user_id in relevant['customer_id_cats']:
        recommendations.append([x[0] for x in qur_model.recommend(userid=user_id, user_items=user_items, N=10)])
    qur_mapk = mapk(relevant['product_id_cats'], recommendations, k=10)
    print(qur_mapk)
    mapk_sum+= qur_mapk
    if qur_mapk > best_mapk:
        best_model = qur_model
        best_mapk = qur_mapk
    model_test.loc[iteration] = [iteration, qur_mapk]
finishTime = time.time()
mapk_mean = mapk_sum / n
meanTime = (finishTime - startTime) / n
print('среднее время обучения:{} секунд'.format(round(meanTime,2)))
print('лучший показатель mapk:',best_mapk)
print('средний показатель mapk:',mapk_mean)
print('__________________________________________________________')
print('модель с максимальным mapk записана в переменной best_model')



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:02<00:22,  2.45s/it]

0.10080854769208204


 20%|██        | 2/10 [00:04<00:19,  2.48s/it]

0.14455607368631046


...

0.0874319310724343


 90%|█████████ | 9/10 [00:25<00:02,  2.93s/it]

0.09459611735490929


100%|██████████| 10/10 [00:28<00:00,  2.95s/it]

0.10482083541866358
среднее время обучения:2.85 секунд
лучший показатель mapk: 0.15792765557359653
средний показатель mapk: 0.1126192723752257
__________________________________________________________
модель с максимальным mapk записана в переменной best_model


In [94]:
# среднее время обучения падает до 2,8 секунд, т.е. в 6 раз
# лучшее значение mapk доходит до 0.158
# среднее значение mapk получается около 0.11

In [95]:
#----------------------------------------------------------------------------------------------

In [96]:
# Добавим в модель информацию о поле посетителей. 
# Суть: 
# сначала была попытка разбить входные данные по полу пользователей и сформировать отдельные по 
# выборки(мужская аудитория, женская и с неизвестным полом), обучить модель в отдельности на каждой 
# из них и поссчитать mapk моделей на разбитых по полу подвыборках тестовой выборки. Результат получился плохой,
# что ожидаемо, но попробовать стоило:)
# Т.к. товары, покупаемые и мужчинами, и женщинами, составляют большую часть, то решил, что данные о поле 
# пользователя буду использовать не для изменения  состава списка наиболее релевантных товаров каждому пользователю, 
# а для их ранжирования. Соответсвенно на основе лучшей обученной модели вывожу перечень наиболее покупаемых товаров 
# в отдельности среди мужчин и среди женщин, формирую 2 списка, далее модель, обученная на всех данных, формирует 
# список 10 товаров для рекомендации каждому пользователю, затем поочередно беру список из этих 10 товаров для 
# пользователя и сравниваю его с соответсвующим списком наиболее востребованных товаров согласно его полу.
# Ранжирование происходит по следующему принципу - из 10 товаров в финальную рекомендацию по очередности идут 
# сначала те, которые входят в список наиболее популярных данного пола(отсортированные согласно изначальной 
# последовательности в рекомендации модели), а затем те, которые в него не входят(аналогично не нарушая порядок 
# следования). Реализуем это и посмотрим что получится.

In [97]:
customers = pd.read_csv('customers.csv')
customers = customers.dropna()
customers = customers.drop_duplicates()

In [98]:
# создаем список наиболее востребованных товаров

In [99]:
customers_products_catcodes_sex = customers_products_catcodes.merge(customers, on = 'customer_id', how = 'left')
customers_products_catcodes_sex.sex = customers_products_catcodes_sex.sex.fillna('NoSex')
customers_sex = customers_products_catcodes_sex[['customer_id_cats', 'sex']]

best_it = best_model.recommend_all(user_items = user_items, N=1)
best_items = pd.DataFrame(best_it)
best_items.reset_index(inplace = True)
best_items.rename(columns = {'index':'customer_id_cats'}, inplace = True)
best_items = best_items.merge(customers_sex, on = 'customer_id_cats', how = 'left')
best_items.head()

,customer_id_cats,0,sex
0,0,551,NoSex
1,1,588,Female
2,2,808,Female
3,3,808,Female
4,4,44,NoSex


In [100]:
# формирую списки для каждого пола в отдельности и общий среди всей выборки.

In [101]:
best_male_items = best_items[best_items.sex == 'Male'][0].unique()
best_female_items = best_items[best_items.sex == 'Female'][0].unique()
best_nosex_items = best_items[best_items.sex == 'NoSex'][0].unique()
best_all_items = best_items[0].unique()

In [102]:
# реализуем функцию, выполняющую ранжирование по задуманной логике

In [103]:
def ranking_recommendations(recommendation, sex):
    if sex == 'Male':
        best_SEX_items = best_male_items
    elif sex == "Female":
        best_SEX_items = best_female_items
    else:
        best_SEX_items = best_all_items
    true_values = []
    false_values = []
    ranked_recommendation = []
    for sub_rec in recommendation:
        if sub_rec in best_SEX_items:
            true_values.append(sub_rec)
        else:
            false_values.append(sub_rec)
    ranked_recommendation.extend(true_values)
    ranked_recommendation.extend(false_values)
    
    return ranked_recommendation

In [104]:
# добавляем в тестовую выборку информацию о поле посетителей

In [105]:
relevant_with_sex = relevant.merge(customers_sex, on = 'customer_id_cats', how = 'left')
relevant_with_sex.head()

,customer_id_cats,product_id_cats,sex
0,0,[5],NoSex
1,1,"[191, 669, 282, 1190, 200, 856, 173, 1091, 10,...",Female
2,2,"[65, 505, 492, 808]",Female
3,3,"[245, 333, 147, 92, 404, 336]",Female
4,6,"[282, 10, 331, 791]",Male


In [106]:
# запускаю серию из N обучений с теми же параметрами, что были в прошлом случае.

In [107]:
best_model = model
best_mapk = 0
best_mapk_new = 0
model_test = pd.DataFrame(columns = ['N_iter', 'mapk_base', 'mapk_new'])
n = 20
mapk_mean = 0
mapk_new_mean = 0
mapk_sum = 0
mapk_new_sum = 0
mapk_new_better_count = 0
for iteration in tqdm(range(n)):
    model = implicit.als.AlternatingLeastSquares(factors=15, iterations=1, calculate_training_loss = True, regularization = 0.01)
    np.random.seed(42)
    model.fit(item_users=item_users)
    qur_model = model
    
    recommendations = []
    for user_id in relevant['customer_id_cats']:
        recommendations.append([x[0] for x in qur_model.recommend(userid=user_id, user_items=user_items, N=10)])
    qur_mapk = mapk(relevant['product_id_cats'], recommendations, k=10)
    
    recommendations_new = []
    for user_id, sex in zip(relevant_with_sex['customer_id_cats'], relevant_with_sex['sex']):
        recommendations_new.append(ranking_recommendations([x[0] for x in qur_model.recommend(userid=user_id, user_items=user_items, N=10)], sex))
    qur_mapk_new = mapk(relevant_with_sex['product_id_cats'], recommendations_new, k=10)
    
    print(qur_mapk, qur_mapk_new)
    mapk_sum+= qur_mapk
    mapk_new_sum += qur_mapk_new
    if qur_mapk > best_mapk:
        best_model = qur_model
        best_mapk = qur_mapk
    if qur_mapk_new > best_mapk_new:
        best_mapk_new = qur_mapk_new
    model_test.loc[iteration] = [iteration, qur_mapk, qur_mapk_new]
    if qur_mapk_new > qur_mapk:
        mapk_new_better_count += 1
mapk_mean = mapk_sum / n
mapk_new_mean = mapk_new_sum / n
print('__________________________________________________________')
print('лучший показатель mapk:',best_mapk)
print('лучший показатель mapk_new:',best_mapk_new)
print('__________________________________________________________')
print('средний показатель mapk:',mapk_mean)
print('средний показатель mapk_new:',mapk_new_mean)
print('__________________________________________________________')
print('улучшение значений mapk в {}% случаев'.format(round((mapk_new_better_count / n * 100), 2)))
print('средний прирост значения mapk составил: {}%'.format(round(((mapk_new_mean - mapk_mean) / mapk_mean * 100), 2)))
print('__________________________________________________________')
print('модель с максимальным mapk записана в переменной best_model')

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:05<01:52,  5.90s/it]

0.08834392554132635 0.08968521511740787


 10%|█         | 2/20 [00:11<01:44,  5.83s/it]

0.13942498126981467 0.13941137777743923


...

0.1263020356148408 0.12638578359082603


 95%|█████████▌| 19/20 [01:48<00:05,  5.65s/it]

0.0884251016687922 0.0886718407115834


100%|██████████| 20/20 [01:54<00:00,  5.65s/it]

0.1361022953636564 0.1379754974653791
__________________________________________________________
лучший показатель mapk: 0.1722586037531551
лучший показатель mapk_new: 0.17228761086560743
__________________________________________________________
средний показатель mapk: 0.11296995745294003
средний показатель mapk_new: 0.11345928343138065
__________________________________________________________
улучшение значений mapk в 75.0% случаев
средний прирост значения mapk составил: 0.43%
__________________________________________________________
модель с максимальным mapk записана в переменной best_model


In [45]:
# Как видим, у переранжированного перечня рекомендаций mapk получается выше в 75-90% обученных моделей,
# а его значение увеличивается в среднем на 1 - 2% относительно рекомендаций модели без доп. ранжирования.
# лучшее значение mapk доходит до 0.17
# среднее значение mapk доходит до 0.113

In [46]:
#-----------------------------------------------------------------------------------------------
# Напоследок небольшой эксперимент - попробуем выявить какое-то количество новой информации о поле посетителей, по
# которым он нам не известен, на основе купленных ими товаров.

In [47]:
customers_products_catcodes_sex.head()

,customer_id,product_id,customer_id_cats,product_id_cats,sex
0,8698595,12530,0,0,NoSex
1,13271885,7541,1,1,Female
2,16852746,13134,2,2,Female
3,14619070,4659,3,4,Female
4,12324374,12072,4,5,NoSex


In [48]:
nosex_customers = customers_products_catcodes_sex[customers_products_catcodes_sex.sex == 'NoSex']

In [49]:
purchases_train_sex = customers_products_catcodes_sex[['customer_id_cats', 'product_id_cats', 'sex']]
purchases_train_sex = purchases_train_sex.drop_duplicates()
purchases_train_sex.head()

,customer_id_cats,product_id_cats,sex
0,0,0,NoSex
1,1,1,Female
2,2,2,Female
3,3,4,Female
4,4,5,NoSex


In [50]:
# создаем список уникальных продуктов для мужчин и для женщин(т.е. товары, которые покупали только мужчины или 
# только женщины)

In [51]:
male_prod = purchases_train_sex[purchases_train_sex.sex == 'Male']['product_id_cats'].unique()
female_prod = purchases_train_sex[purchases_train_sex.sex == 'Female']['product_id_cats'].unique()

In [52]:
male_product = []
for i in male_prod:
    if i not in female_prod:
        male_product.append(i)
female_product = []
for i in female_prod:
    if i not in male_prod:
        female_product.append(i)

In [53]:
len(male_product)

84

In [54]:
len(female_product)

372

In [55]:
nosex_customers.head()

,customer_id,product_id,customer_id_cats,product_id_cats,sex
0,8698595,12530,0,0,NoSex
4,12324374,12072,4,5,NoSex
5,8014926,12072,5,5,NoSex
8,3888648,3072,8,8,NoSex
13,110244,61,13,24,NoSex


In [56]:
# теперь отбираем посетителей, которые покупали товары из уникальных групп товаров

In [57]:
nosex_customers['male_product'] = nosex_customers['product_id_cats'].apply(lambda s: s in male_product)
nosex_customers['female_product'] = nosex_customers['product_id_cats'].apply(lambda s: s in female_product)
nosex_customers.head()

/Users/georabelsky/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/georabelsky/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,customer_id,product_id,customer_id_cats,product_id_cats,sex,male_product,female_product
0,8698595,12530,0,0,NoSex,False,False
4,12324374,12072,4,5,NoSex,False,False
5,8014926,12072,5,5,NoSex,False,False
8,3888648,3072,8,8,NoSex,False,False
13,110244,61,13,24,NoSex,False,False


In [58]:
males_from_nosex = nosex_customers[(nosex_customers.male_product == True) & (nosex_customers.female_product == False)]['customer_id_cats'].unique()
females_from_nosex = nosex_customers[(nosex_customers.male_product == False) & (nosex_customers.female_product == True)]['customer_id_cats'].unique()

In [59]:
# получаем список новых предполагаемых мужчин и женщин

In [60]:
new_males = []
for i in males_from_nosex:
    if i not in females_from_nosex:
        new_males.append(i)

new_females = []
for i in females_from_nosex:
    if i not in males_from_nosex:
        new_females.append(i)

In [61]:
len(new_males)

301

In [62]:
len(new_females)

1093

In [63]:
# получаем новых 301 предполагаемого мужчину и 1093 женщины.

In [64]:
#----------------------------------------------------------------------------------------------